In [ ]:
%matplotlib inline
import matplotlib
import seaborn as sns
matplotlib.rcParams['savefig.dpi'] = 144

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

import expectexception

In [ ]:
sess = None

def reset_tf():
    global sess
    if sess:
        sess.close()
    tf.reset_default_graph()
    sess = tf.Session()

# Optimization Schemes in TensorFlow

Most machine learning algorithms are based on some optimization scheme.  There is a loss function to be minimized, and the parameters of the model are adjusted to decrease the loss.  Most optimization schemes operate iteratively.  They can't find the optimum value right away, but they can improve the target value step by step.

In order to start the next step at the right place, state needs to be stored.  We have already seen two operations that create tensors: constants and placeholders.  Both can be used to enter arbitrary values into a computation through the feed dictionary, but neither offers the ability to hold state.

Consider this (silly) way to calculate $2^{10}$.

In [ ]:
reset_tf()

n = tf.placeholder(np.int64)
n_val = 1
for i in xrange(10):
    n_val = sess.run(2 * n, feed_dict={n: n_val})

print n_val

Because the placeholder doesn't store any state, we had to store the current value of *n* separately, being sure to feed it in correctly at each step.  This is extra work to do, and is obviously inviting errors.  There's a better way to do things.

## Variables

In addition to constants and placeholders, TensorFlow provides **variables**.  These tensors hold values much like others, but these values can be updated during a computation.  We will use variables again and again to hold parameters of models that need to be updated during training.

Variables are created with an initial value.  Like other tensors, they can be given a name and a dtype.

In [ ]:
var = tf.Variable(3, name="my_var", dtype=np.int32)

However, if you try to run a variable, you'll run into a problem:

In [ ]:
%%expect_exception tf.errors.FailedPreconditionError

sess.run(var)

Variables must be initialized before they are run.  The simplest way to do that is with `global_variables_initializer()`.  This TensorFlow operation will initialize all variables.

In [ ]:
init = tf.global_variables_initializer()
sess.run(init)
sess.run(var)

There are many other ways to [initialize variables](https://www.tensorflow.org/programmers_guide/variables#initialization).  Values can be stored on disk and restored later.  You can also initialize only the subset of variables that concern you.  We will use the global initializer so much that we'll make a simple function to run it in our global session:

In [ ]:
def reset_vars():
    sess.run(tf.global_variables_initializer())

Variables don't have to be initialized with a known value. In some cases, we'll want to initialize values randomly.  Here we create a random tensor of shape `[2, 2]` with values drawn from a normal distribution with $\sigma = 1$ and $\mu=2$.

In [ ]:
rand_var = tf.Variable(tf.random_normal([2, 2], stddev=1, mean=2))
reset_vars()
print sess.run(rand_var)

Variables can be updated with the *assign* operation.

In [ ]:
change_var = var.assign(5)

Note that this is an operation.  Nothing changes until the assignment is actually run.

In [ ]:
print sess.run(var)
print sess.run(change_var)
print sess.run(var)

More typically, the value of a variable will be updated based on current values.

In [ ]:
double_var = var.assign(var * 2)
print sess.run(double_var)
print sess.run(double_var)

The value of each variable exists only within a given session.  If you start with another session, variables will have a separate value in that one.

In [ ]:
sess2 = tf.Session()
sess2.run(tf.global_variables_initializer())
print "New session:", sess2.run(var)
print "Old session:", sess.run(var)

### Exercise:

Reimplement the (silly) algorithm from above to calculate $2^{10}$ using a variable instead of a placeholder.

## Gradient descent

One of the simplest optimization schemes is the **gradient descent** algorithm.  For illustration, we'll consider the case where we are optimizing a loss function of a single weight variable $L(W)$.  We want to find $W^*$ such that $L(W^*)$ is a minimum, or equivalently, so $dL(W^*)/dW = 0$.  We don't know the form of $L$, but we can calculate its value and its derivatives at some point $W$.  Thus, we can Taylor expand about $W$ to find

$$ \frac{dL(W^*)}{dW} = 0 = \frac{dL(W)}{dW} + \left(W^* - W\right) \frac{d^2 L(W)}{dW^2} + \ldots $$

Cutting this off after the second derivative, we can approximate the minimum as

$$ W^* = W - \left. \frac{dL}{dW} \middle/ \frac{d^2 L}{dW^2} \right. $$

For gradient descent, we'll assume that we can calculate (or at least approximate) the first derivative, but not the second derivative.  (In the multi-dimensional case, the first derivative becomes the gradient vector and the second derivative becomes the **hessian** matrix, which requires a bit more work to compute.)  Instead, we make use only of the knowledge that near a minimum the second derivative will be positive.  Thus, in gradient descent, the parameters are updated according to

$$ W^* = W - \eta \frac{dL}{dW} $$

where $\eta$ is a constant called the **learning rate**, a hyperparameter.

Gradient descent is a rather straightforward algorithm, but we still probably don't want to implement it ourselves if a case with many parameters.  Luckily, we don't have to: TensorFlow has a number of optimizers built in.  We will demonstrate how to use gradient descent to optimize a linear model on the Boston housing price data.

In [ ]:
import dill
from sklearn.model_selection import ShuffleSplit

with open("small_data/housing.pkl", "r") as fin:
    housing = dill.load(fin)

for train, test in ShuffleSplit(1, 0.2, random_state=42).split(housing.data):
    X_train = housing.data[train]
    y_train = housing.target[train]
    X_test = housing.data[test]
    y_test = housing.target[test]

print X_train.shape

We're going to shift and scale the data, so that all columns are centered about zero and have a standard deviation of 1.  This will help a single learning rate be appropriate for all parameters.

In [ ]:
X_mean = X_train.mean(axis=0)
X_std = X_train.std(axis=0)

Xs_train = (X_train - X_mean) / X_std
Xs_test = (X_test - X_mean) / X_std

print "Mean:", Xs_test.mean()
print "Std: ", Xs_test.std()

We will make placholders for the feature matrix and label vector.  The first dimension will have size `None`, so that we can feed in an arbitrary number of observations.

In [ ]:
reset_tf()

x = tf.placeholder(shape=[None, 13], dtype=tf.float32, name='x')
y_label = tf.placeholder(shape=[None, 1], dtype=tf.float32, name='y_label')

Our linear model is $y = W\cdot x + b$.  Both the weight vector and bias term will be updated by each step of the gradient descent optimizer, so they need to be variables.

In [ ]:
W = tf.Variable(tf.zeros((13, 1)), name="weight")
b = tf.Variable(tf.zeros(1), name="bias")

y = tf.matmul(x, W) + b

We'll choose to minimize the mean squared error as our loss function.

In [ ]:
loss = tf.reduce_mean(tf.square(y - y_label))

When we create an optimizer, we need to specify a learning rate.

In [ ]:
eta = 0.1
optimizer = tf.train.GradientDescentOptimizer(eta)

The optimizer has a `minimize` method.  This method takes a tensor as an argument and returns an operation that will update all the parameters that contribute to this tensor's value.

In [ ]:
train = optimizer.minimize(loss)

This is possible through two pieces of cleverness.

First, by examining the computation graph, the optimizer can find all variables involved in the calculation.  All variables created with the keyword option `trainable=True`, the default, are considered to be parameters that need to be optimized.

Second, through a technique known as [automatic differentiation](https://en.wikipedia.org/wiki/Automatic_differentiation), TensorFlow can exactly calculate numeric values for all derivatives.  This lets the optimizer calculate the gradient of the loss with respect to those parameters and update their values appropriately.

All that's left to do is run the `train` operation several times.  We will feed in the full training set each time.  For evaluation purposes, we'll keep track of both the in- and out-of-sample error.

In [ ]:
in_sample_error = []
out_of_sample_error = []

reset_vars()
for _ in xrange(20):
    sess.run(train, feed_dict={x: Xs_train, y_label: y_train.reshape(-1, 1)})
    in_sample_error.append(sess.run(loss, feed_dict={x: Xs_train, y_label: y_train.reshape(-1, 1)}))
    out_of_sample_error.append(sess.run(loss, feed_dict={x: Xs_test, y_label: y_test.reshape(-1, 1)}))

plt.plot(np.array([in_sample_error, out_of_sample_error]).T, '.-')
plt.legend(['In-sample error', 'Out-of-sample error'])
plt.xlabel('step')
plt.ylabel('loss')

We picked the learning rate more or less at random.  Let's see what happens with different learning rates.

In [ ]:
etas = [0.05, 0.1, 0.15, 0.162, 0.171]
errors = []
BATCH_SIZE = 100

for e in etas:
    reset_vars()
    optimizer = tf.train.GradientDescentOptimizer(e)  # learning rate
    train = optimizer.minimize(loss)
    errors.append([sess.run(loss, feed_dict={x: Xs_train,
                                             y_label: y_train.reshape(-1, 1)})])
    for i in xrange(20):
        sess.run(train, feed_dict={x: Xs_train,
                                   y_label: y_train.reshape(-1,1)})
        errors[-1].append(sess.run(loss, feed_dict={x: Xs_train,
                                                    y_label: y_train.reshape(-1, 1)}))

plt.plot(np.array(errors).T, '.-')
plt.legend(etas, title=r'$\eta$')
plt.xlabel('step')
plt.ylabel('loss');

In this case, the optimal learning rate is around 0.15.  Lower learning rates reach the same solution, but they take longer to do so.  The gradient descent algorithm is taking steps that are too small, causing it to take more steps than strictly necessary.

For larger learning rates, the convergence also slows down.  In this case, the steps are too large.  As each step takes it too far in a direction, it has to waste some of the next step recovering.  As the learning rate goes up, these oversteps become larger and larger, until the algorithm finds itself ending up further away from the optimum each time than it started.  The process no longer converges.

> The learning rate may be chosen by trial and error, but it is usually best to choose it by monitoring learning curves that plot the objective function as a function of time.  This is more of an art than a science, and most guidance on this subject should be regarded with some skepticism.

> &mdash;Ian Goodfellow, Yoshua Bengio, and Aaron Courville, [Deep Learning](http://www.deeplearningbook.org/)

If you get confused by the behavior of your learning curves, don't despair.  You are in [good company](http://lossfunctions.tumblr.com/).

## Stochastic gradient descent

We used the full training set in calculating the loss function above.  With only 404 observations, this is easy enough.  But when dealing with millions of observations (or more), this will become infeasible. It's also unnecessary.  Most observations in a large data set will have many neighbors that impart almost the same information.  We don't need all of them in our learning process.

If we take this reasoning to the extreme, we can train based on the error of a single observation at a time.  We want to train with the "most unexpected" datum each time.  In lieu of trying to figure this out, we'll feed the data randomly, so this process is known as **stochastic gradient descent**.  Each individual step will be noisy, since it contains only a small amount of information.  This means that you will need more steps and a smaller learning rate.

In [ ]:
etas = [0.001, 0.005, 0.02]
errors = []

for e in etas:
    reset_vars()
    optimizer = tf.train.GradientDescentOptimizer(e)
    train = optimizer.minimize(loss)
    errors.append([sess.run(loss, feed_dict={x: Xs_train,
                                                y_label: y_train.reshape(-1, 1)})])
    for i in xrange(1000):
        j = np.random.randint(len(y_train))
        sess.run(train, feed_dict={x: Xs_train[j:j+1,:],
                                   y_label: [[y_train[j]]]})
        errors[-1].append(sess.run(loss, feed_dict={x: Xs_train,
                                                    y_label: y_train.reshape(-1, 1)}))

plt.plot(np.array(errors).T, '-')
plt.legend(etas, title=r'$\eta$')
plt.xlabel('step')
plt.ylabel('loss');

In addition to generally being faster than batch processing, stochastic gradient descent is often more robust.  The noise inherent in this process is often enough to push us out of local minima of the loss function.  (A downside of this noise it that it makes it harder for the model to finally converge at the end.)  Additionally, stochastic gradient descent supports **online learning**.  We can update an existing model with new data just by running some descent steps on it.

## Minibatch processing

**Minibatch processing** attempts to combine the benefits of batch and stochastic gradient descent by training on a "minibatch", a subset of the whole data, at a time.  This increases the accuracy of the gradient calculation, reducing the noise in each step, while still avoiding much of the duplication in the whole set.  Minibatches can often be about as fast as individual steps&mdash;the parallelism available in modern CPUs and especially GPUs is wasted if we are calculating a single row at a time.

A batch size of 100 is usually enough to smooth out the noise.  In our case, the full data is not so much larger than the batch size, so there is little difference in performance.

In [ ]:
etas = [0.05, 0.1, 0.15]
errors = []
BATCH_SIZE = 100

for e in etas:
    reset_vars()
    optimizer = tf.train.GradientDescentOptimizer(e)
    train = optimizer.minimize(loss)
    errors.append([sess.run(loss, feed_dict={x: Xs_train,
                                                y_label: y_train.reshape(-1, 1)})])
    for i in xrange(20):
        j = np.random.choice(len(y_train), BATCH_SIZE, replace=False)
        sess.run(train, feed_dict={x: Xs_train[j, :],
                                   y_label: y_train[j].reshape(-1,1)})
        errors[-1].append(sess.run(loss, feed_dict={x: Xs_train,
                                                    y_label: y_train.reshape(-1, 1)}))

plt.plot(np.array(errors).T, '.-')
plt.legend(etas, title=r'$\eta$')
plt.xlabel('step')
plt.ylabel('loss');

## Logistic Regression

Recall that logistic regression is a linear model used for two-class classification.  Our classes will be labelled 0 and 1, and our model predicts the probability of an observation belonging to class 1 as

$$ p = f\left( X \cdot W + b \right) $$

where $f$ represents the **logistic** (or **sigmoid**) function

$$ f(x) = \frac{1}{1 + e^{-x}} $$

which as a range of $[0, 1]$.

Logistic regression is designed to minimize the cross-entropy:

$$ L = -\frac{1}{n}\sum_{j=1}^{n} y_j \log p_j + (1 - y_j) \log (1-p_j) $$

This expression is more complicated than that of the mean squared error for linear regression, but the same techniques are still available.  We can still take the derivative of this loss function with respect to $W$ and $b$ to work out the update rules in a gradient descent algorithm.  It's even easier than you might think, because TensorFlow provides both the logistic function (as `tf.nn.sigmoid()`) and a entropy calculated from logistic values (as `tf.nn.sigmoid_cross_entropy_with_logits()`).

For an example, we'll classify census tracts on whether their average house price is above 20, in whatever units we happen to be using.

In [ ]:
yc_train = np.float32(y_train > 20)
yc_test = np.float32(y_test > 20)

The setup is nearly identical to the case of linear regression.

In [ ]:
reset_tf()

W = tf.Variable(tf.zeros((13, 1)), name="weight")
b = tf.Variable(tf.zeros(1), name="bias")

x = tf.placeholder(shape=[None, 13], dtype=tf.float32, name='x')
y_label = tf.placeholder(shape=[None, 1], dtype=tf.float32, name='y_label')

y = tf.matmul(x, W) + b
loss = tf.reduce_mean(
    tf.nn.sigmoid_cross_entropy_with_logits(logits=y, labels=y_label))

Here, we make a prediction by calling the logistic function on our linear model, and then comparing that to a threshold of 0.5.  The accuracy is measured by figuring if the predictions are equal to the ground truth labels, casting the result to a float, and taking the average.  Most of us find the accuracy easier to understand than the entropy.

In [ ]:
predicted = tf.cast(tf.nn.sigmoid(y) > 0.5, np.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, y_label), np.float32))

In [ ]:
train = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

reset_vars()

for i in xrange(50):
    j = np.random.choice(len(y_train), BATCH_SIZE, replace=False)
    sess.run(train, feed_dict={x: Xs_train[j,:], 
                               y_label: yc_train[j].reshape(-1,1)})
    if i % 5 == 0:
        print sess.run([loss, accuracy], feed_dict={x: Xs_test, 
                                                    y_label: yc_test.reshape(-1, 1)})

Our model is able to predict 88% of the test data correctly.  Note that in some steps, the entropy improves while the accuracy remains constant.  By measuring our confidence in an answer, the entropy allows improvements even when they don't change any prediction.

## Softmax classification

Recall that any binary classifier can be bootstrapped to multi-class classification with the one-versus-all approach.  If there are $C$ classes, then $C$ different binary classifiers will be built, each classifier $k$ telling class $k$ from not-$k$.

When these scheme is applied to logistic regression, the softmax function is the result.  Let $\pi_k$ be the logit of the probability from the $k^\mathrm{th}$ classifier.  We can represent this in matrix form as

$$ \pi_k^{(j)} = \sum_i X_{ji} W_{ik} + b_k $$

Note that there are no $C$ times as many weights and bias terms.  From these, we can calculate a set of $C$ probabilities

$$ p_k = \frac{e^{-\pi_k}}{\sum_\ell e^{-\pi_\ell}} $$

Because of the chosen normalization, we know that $\sum_k p_k = 1$.  Thus, we can associate $p_k$ with the probability of an observation being in class $k$.

The cross-entropy needs to be expanded to contain a term for each class:

$$ L = -\frac{1}{n}\sum_{j=1}^{n} \sum_{k=1}^C y_k^{(j)} \log p_k^{(j)} $$

Here $y_k^{(j)}$ is an indicator variable which is one if the $j^{th}$ observation is in class $k$, and zero otherwise.  In other words, we are one-hot encoding the labels.

We're going to switch datasets and use the famous [MNIST database](https://en.wikipedia.org/wiki/MNIST_database) of handwritten digits.  Each example is a $28\times28$ pixel grayscale image.  We'll read them in along with one-hot encoded labels.

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets('/tmp/data', one_hot=True)

The images come in as 1-D vectors.  We need to reshape them to view the images.  To get the label, we just look for the index of maximum value of the label vector.

In [ ]:
with sns.axes_style("ticks"):
    plt.imshow(mnist.train.images[0].reshape((28, 28)),
               cmap=plt.cm.gray_r, interpolation='nearest')
print "Label:", mnist.train.labels[0].argmax()

The linear model is very similar to before, but it now has an output of dimension 10, instead of 1.

In [ ]:
reset_tf()

N_PIXELS= 28 * 28
BATCH_SIZE = 100
LEARNING_RATE = 0.5

x = tf.placeholder(tf.float32, [None, N_PIXELS], name="pixels")
y_labels = tf.placeholder(tf.float32, [None, 10], name="labels")

W = tf.Variable(tf.zeros([N_PIXELS, 10]), name="weights")
b = tf.Variable(tf.zeros([10]), name="biases")
y = tf.matmul(x, W) + b

TensorFlow has a built-in softmax entropy function for us to use.  Note that the softmax is monotonic, so we don't need to calculate the probabilities to tell which class we're predicting.  We can just find the largest component of the result of our linear model.

In [ ]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y, 
                                                              labels=y_labels))
train = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(y, 1),
                                           tf.argmax(y_labels, 1)),
                                  tf.float32))

If we run the optimizer, we can pretty easily manage about 92% accuracy.  This is not bad, given the simplicity of the model.  Random guessing would have only given us 10% accuracy.

In [ ]:
reset_vars()

for i in range(1000):
    batch_x, batch_y = mnist.train.next_batch(BATCH_SIZE)
    sess.run(train,
             feed_dict={x: batch_x, y_labels: batch_y})
    if i % 100 == 0:
        print sess.run([loss, accuracy],
                       feed_dict={x: mnist.test.images, y_labels: mnist.test.labels})

### Exercises

1. Adjust the learning rate and the batch size.  Can you get better performance?

2. The weight matrix $W$ contains information about how much each pixel contributes to each class.  Plot these contributions for several of the numbers.  Can you understand what the model was looking for in each case?

3. The [Iris data set](https://en.wikipedia.org/wiki/Iris_flower_data_set) is famous set of measurements of four features from three different species of *Iris*.  Build a classifer to descriminate between the three species based on these four features.  The data are provided by Scikit-Learn:

In [ ]:
from sklearn import datasets
iris = datasets.load_iris()
print "Features:", iris.data.shape
print "Labels:", iris.target.shape

*Hint:* The labels are just integers specifying the class.  You will probably need to convert them to a one-hot encoded value; [`tf.one_hot()`](https://www.tensorflow.org/versions/master/api_docs/python/tf/one_hot) might be useful.

*Copyright &copy; 2017 The Data Incubator.  All rights reserved.*